In [1]:
library(edgeR)

Warning message:
“package ‘edgeR’ was built under R version 4.1.2”
Loading required package: limma

Warning message:
“package ‘limma’ was built under R version 4.1.2”


In [2]:
clusts = unlist(read.csv('../../data/ref/cluster_list.txt', header=FALSE)[[1]])

In [3]:
clusts

[1] "ASC"        "ChC"        "Endo"       "L2_3_IT"    "L4_5_IT"   
 [6] "L5_6_NP"    "L5_ET"      "L5_IT"      "L6b"        "L6_CT"     
[11] "L6_IT_CAR3" "L6_IT"      "LAMP5"      "MGC"        "ODC"       
[16] "OPC"        "PVALB"      "SNCG"       "SST"        "VIP"       
[21] "VLMC"

In [4]:
sample_species =  unlist(read.csv('../../data/processed/gex_for_edger/sample_order_ASC.txt', header=FALSE)[[1]])
species = unique(sample_species)

In [5]:
species

[1] "human"    "macaque"  "marmoset" "mouse"

In [6]:
species_idx = c("human" = 1, "macaque" = 2, "marmoset" = 3, "mouse" = 4)

In [7]:
count_path = '../../data/processed/gex_for_edger/%s_all_species_orthologous_counts.txt'

In [8]:
exclude_genes =  unlist(read.csv('../../data/ref/missing_annot_in_species.txt', header=FALSE)[[1]])

# lrt

In [9]:
one_v_rest = c(-1/3, -1/3, -1/3, -1/3)
for (clust in clusts){
    sample_species =  unlist(read.csv(sprintf('../../data/processed/gex_for_edger/sample_order_%s.txt', clust), header=FALSE)[[1]])
    path_use = sprintf(count_path, clust)
    counts = read.table(path_use, sep='\t', row.names=1)
    use_genes = rowSums(is.na(counts)) == 0
    counts = counts[use_genes, ]
    use  = colSums(is.na(counts)) == 0
    use_info = sample_species[use]
    counts = counts[, use]
    dge = DGEList(counts=counts, group=use_info)
    keep = filterByExpr(dge)
    dge = dge[keep, ,keep.lib.sizes=FALSE]
    dge = calcNormFactors(dge, 'TMM')
    dge = estimateDisp(dge, trend.method='locfit')
    design = model.matrix(~0 + dge$samples$group)
    fit = glmFit(dge, design = design)
    print(clust)
    for (specie in species){
        one_v_rest = c(-1/3, -1/3, -1/3, -1/3)
        one_v_rest[species_idx[specie]] = 1  
        vrest = glmLRT(fit, contrast=one_v_rest)
        write.table(topTags(vrest, n = "Inf")$table, file=sprintf("../../data/outputs/gex_de_results/lrt/%s_v_all_%s.tsv", specie, clust), sep="\t")
        # print(specie)
        # print(sum(topTags(vrest, n = "Inf")$table$FDR <= .0125))
         for (specie_2 in species){
            if (specie_2 == specie){
                next
            }
            v1 = c(0, 0, 0, 0)
            v1[species_idx[specie]] = 1
            v1[species_idx[specie_2]] = -1
            v1_test = glmLRT(fit, contrast=v1)
            # print(sum(topTags(v1_test, n = "Inf")$table$FDR <= .0125))
            write.table(topTags(v1_test, n = "Inf")$table, file=sprintf("../../data/outputs/gex_de_results/lrt/%s_v_%s_%s.tsv", specie, specie_2, clust), sep="\t")
        }
    }
}

Using classic mode.



[1] "ASC"


Using classic mode.



[1] "ChC"


Using classic mode.



...

[1] "SST"


Using classic mode.



[1] "VIP"


Using classic mode.



[1] "VLMC"


In [11]:
print('done')

[1] "done"
